# Machine Learning Getting Started Notebook

This notebook demonstrates basic machine learning workflows using scikit-learn.

## Sections:
1. Data Loading and Exploration
2. Data Preprocessing
3. Model Training
4. Model Evaluation
5. Visualization

In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris, load_boston
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

print("Libraries imported successfully!")

ImportError: 
`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[:

## 1. Data Loading and Exploration

In [ ]:
# Load the Iris dataset
iris = load_iris()
X, y = iris.data, iris.target

# Create a DataFrame for easier manipulation
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y
df['species'] = df['target'].map({0: 'setosa', 1: 'versicolor', 2: 'virginica'})

print(f"Dataset shape: {df.shape}")
print("\nFirst 5 rows:")
df.head()

In [ ]:
# Basic statistics
print("Dataset Info:")
print(df.info())
print("\nStatistical Summary:")
df.describe()

## 2. Data Visualization

In [ ]:
# Create visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Pairplot
sns.pairplot(df, hue='species', diag_kind='hist')
plt.suptitle('Iris Dataset Pairplot', y=1.02)
plt.show()

# Correlation heatmap
plt.figure(figsize=(10, 8))
correlation_matrix = df.select_dtypes(include=[np.number]).corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Feature Correlation Matrix')
plt.show()

## 3. Data Preprocessing and Model Training

In [ ]:
# Prepare the data
X = iris.data
y = iris.target

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")

In [ ]:
# Train multiple models
models = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42)
}

results = {}

for name, model in models.items():
    # Train the model
    if name == 'Logistic Regression':
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    results[name] = {'model': model, 'accuracy': accuracy, 'predictions': y_pred}
    
    print(f"{name} Accuracy: {accuracy:.4f}")

print("\nModel training completed!")

## 4. Model Evaluation

In [ ]:
# Detailed evaluation for the best model
best_model_name = max(results.keys(), key=lambda k: results[k]['accuracy'])
best_predictions = results[best_model_name]['predictions']

print(f"Best Model: {best_model_name}")
print(f"Best Accuracy: {results[best_model_name]['accuracy']:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, best_predictions, target_names=iris.target_names))

# Confusion Matrix
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, best_predictions)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.title(f'Confusion Matrix - {best_model_name}')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

## 5. Feature Importance (Random Forest)

In [ ]:
# Feature importance for Random Forest
if 'Random Forest' in results:
    rf_model = results['Random Forest']['model']
    feature_importance = rf_model.feature_importances_
    
    # Create feature importance plot
    plt.figure(figsize=(10, 6))
    indices = np.argsort(feature_importance)[::-1]
    
    plt.bar(range(len(feature_importance)), feature_importance[indices])
    plt.xticks(range(len(feature_importance)), [iris.feature_names[i] for i in indices], rotation=45)
    plt.title('Feature Importance - Random Forest')
    plt.ylabel('Importance Score')
    plt.tight_layout()
    plt.show()
    
    print("Feature Importance Ranking:")
    for i, idx in enumerate(indices):
        print(f"{i+1}. {iris.feature_names[idx]}: {feature_importance[idx]:.4f}")

## Summary

This notebook demonstrated:
- Data loading and exploration
- Data visualization techniques
- Data preprocessing (scaling)
- Training multiple ML models
- Model evaluation and comparison
- Feature importance analysis

Key takeaways:
- Always explore your data before modeling
- Feature scaling can improve model performance
- Compare multiple models to find the best one
- Use appropriate evaluation metrics for your task